In [2]:
import src.config as config

In [5]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date = }')

fetch_date_to = current_date

fetch_date_from = current_date - timedelta(days = 28)

current_date = Timestamp('2024-12-29 04:00:00')


In [6]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    from_date_ = from_date - timedelta(days = 7*52)
    to_date_ = to_date - timedelta(days = 7*52)

    rides = load_raw_data(year = from_date_.year, months = from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]

    rides_2 = load_raw_data(year = to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]
    rides = pd.concat([rides, rides_2])

    rides["pickup_datetime"] += timedelta(days=7*52)

    rides.sort_values(by = ["pickup_location_id", "pickup_datetime"], inplace = True)
    return rides

In [7]:
rides = fetch_batch_raw_data(from_date=fetch_date_from, to_date=fetch_date_to)

 File 2023-12 already exists
 File 2023-12 already exists


In [11]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 259/259 [00:00<00:00, 399.12it/s]


In [13]:
import hopsworks

project = hopsworks.login(
    project = config.HOPSWORK_PROJECT_NAME,
    api_key_value = config.HOPSWORK_API_KEY
)

feature_store = project.get_feature_store()

feature_group = feature_store.get_or_create_feature_group(
    name = config.FEATURE_GROUP_NAME,
    version = config.FEATURE_GROUP_VERSION,
    description = "Time-series data at hourly frequency",
    primary_key = ["pickup_location_id","pickup_hour"],
    event_time = "pickup_hour"
)

2024-12-29 10:24:01,391 INFO: Initializing external client
2024-12-29 10:24:01,391 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-29 10:24:07,186 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207467


In [16]:
ts_data['pickup_location_id'] = ts_data['pickup_location_id'].astype('int64')

print(ts_data.dtypes)

pickup_hour           datetime64[ns]
rides                        float64
pickup_location_id             int64
dtype: object


In [17]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 192696/192696 | Elapsed Time: 00:12 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207467/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)